In [1]:
import numpy as np
import pandas as pd

In [4]:
# df_healthy = pd.read_csv(
#     "/Users/maxverbiest/PhD/projects/genotype_strs/results/oxana_variation/crc_bloodnormal_18may.csv",
#     sep=","
# ).drop("sample_type", axis=1)

# df_tumor = pd.read_csv(
#     "/Users/maxverbiest/PhD/projects/genotype_strs/results/oxana_variation/crc_primarytumor_19may.csv",
#     sep=","
# ).drop("sample_type", axis=1)

/var/folders/2z/jqp3h4fs3lsfdqvrjxl783f00000gn/T/ipykernel_43082/2042205938.py:1: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  df_healthy = pd.read_csv(


In [14]:
df_healthy = pd.read_csv(
    "/Users/maxverbiest/PhD/projects/genotype_strs/results/oxana_variation/crc_solid_april_stn.csv",
    sep=","
).drop("sample_type", axis=1)

df_tumor = pd.read_csv(
    "/Users/maxverbiest/PhD/projects/genotype_strs/results/oxana_variation/crc_solid_april_pt.csv",
    sep=","
).drop("sample_type", axis=1)

In [17]:
df_tumor = df_tumor.drop_duplicates()
df_tumor

,patient,sample_type,repeat_id,chr,start,end,period,ref,alt,tmp_id
0,TCGA-AA-3489,Primary Tumor,1099253.0,chr1,187770,187777,2,4,4,chr1_187770
1,TCGA-AA-3489,Primary Tumor,1099308.0,chr1,948123,948130,2,4,4,chr1_948123
2,TCGA-AA-3489,Primary Tumor,1099283.0,chr1,971311,971322,4,3,3,chr1_971311
3,TCGA-AA-3489,Primary Tumor,1099358.0,chr1,1043493,1043500,2,4,4,chr1_1043493
4,TCGA-AA-3489,Primary Tumor,1099359.0,chr1,1043875,1043884,2,5,5,chr1_1043875
...,...,...,...,...,...,...,...,...,...,...
850366,TCGA-AA-3514,Primary Tumor,888217.0,chrX,154532321,154532332,4,3,3,chrX_154532321
850367,TCGA-AA-3514,Primary Tumor,888228.0,chrX,154618918,154618952,5,7,6,chrX_154618918
850368,TCGA-AA-3514,Primary Tumor,888231.0,chrX,154653227,154653241,5,3,3,chrX_154653227
850369,TCGA-AA-3514,Primary Tumor,888313.0,chrX,154762860,154762871,4,3,3,chrX_154762860


In [18]:
df_tumor.to_csv("../../results/oxana_variation/crc_solid_april_pt_uniq.csv", header=True, index=False)

In [46]:
def to_wide_df(df: pd.DataFrame, sample_type: str) -> pd.DataFrame:
    if not sample_type in {"healthy", "tumor"}:
        raise ValueError("Sample type must be either 'healthy' or 'tumor'")
    df_to_dupe = (
        df[["patient", "tmp_id", "alt"]]
            .groupby(["patient", "tmp_id"])
            .filter(lambda x: len(x) == 1)
    )
    df_wide = (
        pd.concat([df_to_dupe, df[["patient", "tmp_id", "alt"]]])
            .sort_values(by=["patient", "tmp_id"])
            .reset_index(drop=True)
    )
    df_wide["allele"] = [f'allele_a_{sample_type}', f'allele_b_{sample_type}'] * int((df_wide.shape[0] / 2))
    df_wide = (
        df_wide
            .pivot(index=["patient", "tmp_id"], columns="allele", values="alt")
            .reset_index()
            .merge(df.drop(["patient", "alt"], axis=1).drop_duplicates(), how="left", on="tmp_id")
    )
    
    return df_wide
    

In [47]:
%%time
df_healthy_wide = to_wide_df(df_healthy, sample_type="healthy")
df_healthy_wide

CPU times: user 1.98 s, sys: 36.1 ms, total: 2.02 s
Wall time: 2.02 s


,patient,tmp_id,allele_a_healthy,allele_b_healthy,repeat_id,chr,start,end,period,ref
0,TCGA-AA-3966,chr10_100042463,3,3,223515.0,chr10,100042463,100042474,4,3
1,TCGA-AA-3966,chr10_100048963,14,14,223550.0,chr10,100048963,100048976,1,14
2,TCGA-AA-3966,chr10_100057216,7,7,223501.0,chr10,100057216,100057229,2,7
3,TCGA-AA-3966,chr10_1000593,4,4,166727.0,chr10,1000593,1000600,2,4
4,TCGA-AA-3966,chr10_100063891,5,5,NaN,chr10,100063891,100063900,2,5
...,...,...,...,...,...,...,...,...,...,...
66960,TCGA-AA-3966,chrX_97247622,9,9,867725.0,chrX,97247622,97247630,1,9
66961,TCGA-AA-3966,chrX_97247846,3,3,868395.0,chrX,97247846,97247857,4,3
66962,TCGA-AA-3966,chrX_97429811,10,10,NaN,chrX,97429811,97429822,1,12
66963,TCGA-AA-3966,chrX_9760652,4,4,835111.0,chrX,9760652,9760663,3,4


In [48]:
%%time
df_tumor_wide = to_wide_df(df_tumor, sample_type="tumor")
df_tumor_wide

CPU times: user 1.86 s, sys: 43.5 ms, total: 1.9 s
Wall time: 1.9 s


,patient,tmp_id,allele_a_tumor,allele_b_tumor,repeat_id,chr,start,end,period,ref
0,TCGA-AA-3966,chr10_100042463,3,3,223515.0,chr10,100042463,100042474,4,3
1,TCGA-AA-3966,chr10_100048963,14,14,223550.0,chr10,100048963,100048976,1,14
2,TCGA-AA-3966,chr10_100056718,13,13,223546.0,chr10,100056718,100056731,1,14
3,TCGA-AA-3966,chr10_100057216,7,7,223501.0,chr10,100057216,100057229,2,7
4,TCGA-AA-3966,chr10_1000593,4,4,166727.0,chr10,1000593,1000600,2,4
...,...,...,...,...,...,...,...,...,...,...
67920,TCGA-AA-3966,chrX_9716220,4,4,834724.0,chrX,9716220,9716227,2,4
67921,TCGA-AA-3966,chrX_97247622,9,9,867725.0,chrX,97247622,97247630,1,9
67922,TCGA-AA-3966,chrX_97247846,3,3,868395.0,chrX,97247846,97247857,4,3
67923,TCGA-AA-3966,chrX_97429811,10,10,NaN,chrX,97429811,97429822,1,12


In [67]:
df_merged_wide = df_healthy_wide.merge(
    df_tumor_wide[["patient", "tmp_id", "allele_a_tumor", "allele_b_tumor"]],
    how="left",
    on=["patient", "tmp_id"]
).dropna(subset=["allele_a_tumor", "allele_b_tumor"], axis=0)

df_merged_wide

,patient,tmp_id,allele_a_healthy,allele_b_healthy,repeat_id,chr,start,end,period,ref,allele_a_tumor,allele_b_tumor
0,TCGA-AA-3966,chr10_100042463,3,3,223515.0,chr10,100042463,100042474,4,3,3.0,3.0
1,TCGA-AA-3966,chr10_100048963,14,14,223550.0,chr10,100048963,100048976,1,14,14.0,14.0
2,TCGA-AA-3966,chr10_100057216,7,7,223501.0,chr10,100057216,100057229,2,7,7.0,7.0
3,TCGA-AA-3966,chr10_1000593,4,4,166727.0,chr10,1000593,1000600,2,4,4.0,4.0
4,TCGA-AA-3966,chr10_100063891,5,5,NaN,chr10,100063891,100063900,2,5,5.0,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...
66959,TCGA-AA-3966,chrX_9716220,4,4,834724.0,chrX,9716220,9716227,2,4,4.0,4.0
66960,TCGA-AA-3966,chrX_97247622,9,9,867725.0,chrX,97247622,97247630,1,9,9.0,9.0
66961,TCGA-AA-3966,chrX_97247846,3,3,868395.0,chrX,97247846,97247857,4,3,3.0,3.0
66962,TCGA-AA-3966,chrX_97429811,10,10,NaN,chrX,97429811,97429822,1,12,10.0,10.0


In [71]:
# reorder columns
df_merged_wide = df_merged_wide[['tmp_id', 'repeat_id', 'chr', 'start', 'end', 'period', 'ref', 'patient', 'allele_a_healthy', 'allele_b_healthy', 'allele_a_tumor','allele_b_tumor']]

# repeat_id column should also be int, but it contains 'NA' values so conversion does not work
int_cols = ["start", "end", "period", "ref", "allele_a_healthy", "allele_b_healthy", "allele_a_tumor", "allele_b_tumor"]
df_merged_wide[int_cols] = df_merged_wide[int_cols].astype(int)

df_merged_wide

,tmp_id,repeat_id,chr,start,end,period,ref,patient,allele_a_healthy,allele_b_healthy,allele_a_tumor,allele_b_tumor
0,chr10_100042463,223515.0,chr10,100042463,100042474,4,3,TCGA-AA-3966,3,3,3,3
1,chr10_100048963,223550.0,chr10,100048963,100048976,1,14,TCGA-AA-3966,14,14,14,14
2,chr10_100057216,223501.0,chr10,100057216,100057229,2,7,TCGA-AA-3966,7,7,7,7
3,chr10_1000593,166727.0,chr10,1000593,1000600,2,4,TCGA-AA-3966,4,4,4,4
4,chr10_100063891,NaN,chr10,100063891,100063900,2,5,TCGA-AA-3966,5,5,5,5
...,...,...,...,...,...,...,...,...,...,...,...,...
66959,chrX_9716220,834724.0,chrX,9716220,9716227,2,4,TCGA-AA-3966,4,4,4,4
66960,chrX_97247622,867725.0,chrX,97247622,97247630,1,9,TCGA-AA-3966,9,9,9,9
66961,chrX_97247846,868395.0,chrX,97247846,97247857,4,3,TCGA-AA-3966,3,3,3,3
66962,chrX_97429811,NaN,chrX,97429811,97429822,1,12,TCGA-AA-3966,10,10,10,10
